# Evaluation Metrics

Evaluate self-recognition, model attribution, and cross model detection performance.


In [ ]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Paths
BASE_DIR = Path("/Users/ehsan/CursorProjects/llm-collusion")
SELF_RECOGNITION_FILE = BASE_DIR / "data" / "attribution" / "self_recognition" / "mbpp-sanitized" / "test" / "anthropic-claude-haiku-4.5.jsonl"
MODEL_ATTRIBUTION_DIR = BASE_DIR / "data" / "attribution" / "model_attribution" / "mbpp-sanitized" / "test"
CROSS_MODEL_DETECTION_DIR = BASE_DIR / "data" / "attribution" / "cross_model_detection" / "mbpp-sanitized" / "test"


In [2]:
# Self-Recognition Metrics
print("=" * 60)
print("SELF-RECOGNITION")
print("=" * 60)

data = [json.loads(line) for line in open(SELF_RECOGNITION_FILE) if line.strip()]
df_sr = pd.DataFrame(data)

y_true = df_sr['gold_candidate'].values
y_pred = df_sr['predicted_candidate'].values

acc = accuracy_score(y_true, y_pred)
precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro', zero_division=0)

print(f"Accuracy:  {acc:.4f} ({acc*100:.2f}%)")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"Correct:   {df_sr['is_correct'].sum()}/{len(df_sr)}")


SELF-RECOGNITION
Accuracy:  0.5097 (50.97%)
Precision: 0.5094
Recall:    0.5094
F1-Score:  0.5094
Correct:   131/257


In [3]:
# Model Attribution Metrics
print("\n" + "=" * 60)
print("MODEL ATTRIBUTION")
print("=" * 60)

def extract_attribution_labels(records):
    """Extract labels from model attribution records."""
    y_true, y_pred = [], []
    
    if not records:
        return y_true, y_pred
    
    # Get model names
    model1 = records[0].get("model1")
    model2 = records[0].get("model2")
    if not model1 or not model2:
        return y_true, y_pred
    
    for r in records:
        gold = r.get("gold_attribution")
        pred = r.get("predicted_attribution")
        if not gold or not pred:
            continue
        
        # Code1
        g1 = gold.get("Code1")
        p1 = pred.get("Code1")
        if g1 in (model1, model2) and p1 in (model1, model2):
            y_true.append(0 if g1 == model1 else 1)
            y_pred.append(0 if p1 == model1 else 1)
        
        # Code2
        g2 = gold.get("Code2")
        p2 = pred.get("Code2")
        if g2 in (model1, model2) and p2 in (model1, model2):
            y_true.append(0 if g2 == model1 else 1)
            y_pred.append(0 if p2 == model1 else 1)
    
    return y_true, y_pred

# Load all model attribution files
attribution_files = list(MODEL_ATTRIBUTION_DIR.glob("*.jsonl"))
all_y_true, all_y_pred = [], []

for file in attribution_files:
    records = [json.loads(line) for line in open(file) if line.strip()]
    y_true, y_pred = extract_attribution_labels(records)
    all_y_true.extend(y_true)
    all_y_pred.extend(y_pred)

if all_y_true:
    acc = accuracy_score(all_y_true, all_y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(all_y_true, all_y_pred, average='macro', zero_division=0)
    
    print(f"Accuracy:  {acc:.4f} ({acc*100:.2f}%)")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"Support:   {len(all_y_true)} code items")
    print(f"Files:     {len(attribution_files)}")
else:
    print("No attribution data found")



MODEL ATTRIBUTION
Accuracy:  0.6555 (65.55%)
Precision: 0.6555
Recall:    0.6555
F1-Score:  0.6555
Support:   1016 code items
Files:     2


In [4]:
# Cross Model Detection Metrics
print("\n" + "=" * 60)
print("CROSS MODEL DETECTION")
print("=" * 60)

# Load all cross model detection files
detection_files = list(CROSS_MODEL_DETECTION_DIR.glob("*.jsonl"))
all_y_true_cmd, all_y_pred_cmd = [], []

for file in detection_files:
    records = [json.loads(line) for line in open(file) if line.strip()]
    df_cmd = pd.DataFrame(records)
    # Filter out rows with NaN values
    df_cmd_clean = df_cmd.dropna(subset=['gold_candidate', 'predicted_candidate'])
    if len(df_cmd_clean) > 0:
        y_true = df_cmd_clean['gold_candidate'].values
        y_pred = df_cmd_clean['predicted_candidate'].values
        all_y_true_cmd.extend(y_true)
        all_y_pred_cmd.extend(y_pred)

if all_y_true_cmd:
    # Convert to numpy arrays and ensure no NaN
    all_y_true_cmd = np.array(all_y_true_cmd)
    all_y_pred_cmd = np.array(all_y_pred_cmd)
    
    acc = accuracy_score(all_y_true_cmd, all_y_pred_cmd)
    precision, recall, f1, _ = precision_recall_fscore_support(all_y_true_cmd, all_y_pred_cmd, average='macro', zero_division=0)
    
    print(f"Accuracy:  {acc:.4f} ({acc*100:.2f}%)")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"Support:   {len(all_y_true_cmd)} records")
    print(f"Files:     {len(detection_files)}")
else:
    print("No cross model detection data found")



CROSS MODEL DETECTION
Accuracy:  0.9623 (96.23%)
Precision: 0.9630
Recall:    0.9630
F1-Score:  0.9623
Support:   769 records
Files:     3
